In [33]:
import os
import json

In [36]:
import requests
import time

In [30]:
__location__ = os.path.realpath( os.path.join(os.getcwd(), os.path.dirname(__file__)))

NameError: name '__file__' is not defined

In [118]:
import pickle, csv, shutil

In [119]:
class PersistentData(dict):
    ''' Sync data between mem and disk
    '''

    def __init__(self, filename, flag='c', mode=None, format='pickle', *args, **kwds):
        self.flag = flag                    # r=readonly, c=create, or n=new
        self.mode = mode                    # None or an octal triple like 0644
        self.format = format                # 'csv', 'json', or 'pickle'
        self.filename = filename
        if flag != 'n' and os.access(filename, os.R_OK):
            fileobj = open(filename, 'rb' if format == 'pickle' else 'r')
            with fileobj:
                self.load(fileobj)
        dict.__init__(self, *args, **kwds)

    def sync(self):
        '''
        Write the dictionary to disk
        '''
        if self.flag == 'r':
            return
        filename = self.filename
        tempname = filename + '.tmp'
        fileobj = open(tempname, 'wb' if self.format == 'pickle' else 'w')
        try:
            self.dump(fileobj)
        except Exception:
            os.remove(tempname)
            raise
        finally:
            fileobj.close()
        shutil.move(tempname, self.filename)    # atomic commit
        if self.mode is not None:
            os.chmod(self.filename, self.mode)

    def close(self):
        '''
        Write the dictionary to the corresponding file on disk
        '''
        self.sync()

    def __enter__(self):
        return self

    def __exit__(self, *exc_info):
        self.close()

    def dump(self, fileobj):
        if self.format == 'csv':
            csv.writer(fileobj).writerows(self.items())
        elif self.format == 'json':
            json.dump(self, fileobj, indent=2, separators=(',', ':'))
        elif self.format == 'pickle':
            pickle.dump(dict(self), fileobj, 2)
        else:
            raise NotImplementedError('Unknown format: ' + repr(self.format))

    def load(self, fileobj):
        # try formats from most restrictive to least restrictive
        for loader in (pickle.load, json.load, csv.reader):
            fileobj.seek(0)
            try:
                return self.update(loader(fileobj))
            except Exception:
                pass
        raise ValueError('File not in a supported format')

In [120]:
data = PersistentData('data.json', format='json')

In [89]:
url_base = 'https://mars.nasa.gov/rss/api/?feed=weather&category=insight&feedtype=json&ver=1.0'

In [90]:
def retry_timer(which_retry, retry_base_interval, mode = None):
    """Calculate a random retry interval

    Args:
        mode(optional, default=None): specify the mode of retry time
            list of possible values: 'random', 'multiply', 'multirand'
    """

    if mode == None:
        mode = 'random'

    if mode == 'random':
        retry_wait_interval = retry_base_interval * random.random()
    elif mode == 'multiply':
        retry_wait_interval = which_retry * retry_base_interval
    elif mode == 'multirand':
        retry_wait_interval = which_retry * retry_base_interval * random.random()

    return {'mode': mode, 'interval': retry_wait_interval, 'retry': which_retry }

In [91]:
def get_remote(url_inp):
    
    
    which_retry = 0

    while True and (which_retry < 10):
        
        try:
            resp = requests.get(url_inp)
            if resp.status_code == 200:
                return resp.json()
        except Exception as e:
            print('Failed to request from url {}, status code: {}. Retrying'.format(url_inp, resp.status_code), e )
            which_retry = which_retry + 1
            retry_sleep_time = _retry_timer( which_retry, 1, mode = 'multirand' ).get('interval')
            print('Retry ({}) in {} seconds.'.format(which_retry, retry_sleep_time))
            sleep(retry_sleep_time)
            pass
    


In [92]:
get_remote(url_base)

{'75': {'AT': {'av': -62.746, 'ct': 17755, 'mn': -94.311, 'mx': -9.607},
  'First_UTC': '2019-02-11T06:39:54Z',
  'HWS': {'av': 4.561, 'ct': 8171, 'mn': 0.186, 'mx': 12.628},
  'Last_UTC': '2019-02-12T07:19:28Z',
  'PRE': {'av': 720.154, 'ct': 276913, 'mn': 686.234, 'mx': 744.424},
  'Season': 'winter',
  'WD': {'0': {'compass_degrees': 0.0,
    'compass_point': 'N',
    'compass_right': 0.0,
    'compass_up': 1.0,
    'ct': 179},
   '1': {'compass_degrees': 22.5,
    'compass_point': 'NNE',
    'compass_right': 0.382683432365,
    'compass_up': 0.923879532511,
    'ct': 42},
   '10': {'compass_degrees': 225.0,
    'compass_point': 'SW',
    'compass_right': -0.707106781187,
    'compass_up': -0.707106781187,
    'ct': 1793},
   '11': {'compass_degrees': 247.5,
    'compass_point': 'WSW',
    'compass_right': -0.923879532511,
    'compass_up': -0.382683432365,
    'ct': 284},
   '12': {'compass_degrees': 270.0,
    'compass_point': 'W',
    'compass_right': -1.0,
    'compass_up': -0.0

In [114]:
def clean_up_response(rsp_inp, loaded_data = None):
    
    if loaded_data is None:
        loaded_data = {}
    
    sol_keys = rsp_inp.get('sol_keys')

    if sol_keys is not None:
        for key in sol_keys:
            loaded_data[key] = rsp_inp.get(key)
    
    return loaded_data

In [115]:
data = clean_up_response(get_remote(url_base), loaded_data=data)

In [116]:
data.sync()

In [117]:
data.keys()

dict_keys(['76', '75', '77', '78', '79', '80', '81'])